# EWC

In [23]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adamax
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical

In [2]:
import import_ipynb
import pandas as pd
import numpy as np

In [3]:
from copy import deepcopy

In [4]:
import utils
from utils import *

In [12]:
import Model 
from Model import *

In [44]:
def compute_ewc_penalty(model, fisher_matrix, optimal_weights, lamb):   
    loss = 0
    current = model.trainable_weights 
    
    for F, c, o in zip(fisher_matrix, current, optimal_weights):
        loss += tf.reduce_sum(F * ((c - o) ** 2))


    return loss * (lamb / 2)

In [6]:
def ewc_loss(model, fisher_matrix, lamb):
    optimal_weights = deepcopy(model.trainable_weights)

    def loss_fn(y_true, y_pred):

        ce_loss = CategoricalCrossentropy(from_logits=False)(y_true, y_pred)
        ewc_loss = compute_ewc_penalty(model, fisher_matrix, optimal_weights, lamb)

        return ce_loss + ewc_loss
    
    return loss_fn

In [89]:
def compute_fisher_matrix(model, data, num_sample=10):

    weights = model.trainable_weights
    variance = [tf.zeros_like(tensor) for tensor in weights]

    # num_sample 개의 데이터 랜덤샘플링 
    indices = np.random.choice(len(data), size=num_sample, replace=False)

    for i in indices:

        with tf.GradientTape() as tape:
            tape.watch(weights)
            x = tf.expand_dims(data[i], axis=0)
            output = model(x, training=False) # (수정) 메모리 문제, 모든 데이터를 한번에 넣으면 오류 생김. 여기서는 하나씩 열개의 데이터를 사용 
            log_likelihood = tf.math.log(output)

        gradients = tape.gradient(log_likelihood, weights)
        variance = [var + (grad ** 2) for var, grad in zip(variance, gradients)]

    fisher_matrix = [tensor / num_sample for tensor in variance]
    
    return fisher_matrix

In [65]:
# (수정) 배치를 사용하지 않음 
def evaluate(model, test_set):
  acc = tf.keras.metrics.CategoricalAccuracy(name='accuracy')
  for i, (seq, labels) in enumerate(test_set):
    preds = model.predict_on_batch(seq)
    acc.update_state(labels, preds)
  return acc.result().numpy()

In [95]:

def train_loop(model, OPTIMIZER, data, test_size,
                first_task = 44, inc_task = 5, first_epochs = 30, inc_epochs = 5,
                  lamb=0, num_sample=10):
    
    first_part = split_by_label(data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) <= MAX_LABEL:
            
            if i == 0:
                model.compile(loss=CategoricalCrossentropy(from_logits=False), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)
                

                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=1)
                print(f"   First_task training accuracy: {history.history['accuracy'][-1]:.4f}")

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)

                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb), optimizer=OPTIMIZER, metrics=["accuracy"])
                
                # 3D ndarray 로 변환 (이부분 함수로 바꾸기)
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)



                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=1)
                print(f"   {i}_task training accuracy: {history.history['accuracy'][-1]:.4f}")



                # (수정) 일종의 전처리이므로 preprocessing 또는 utils에 함수 작성 
                # 축적된 test로 정확도 측정 (중요, EWC 성능)
                test_seq, test_label = split_data_label(test)
                test_seq = np.stack(test_seq.values)
                test_seq = test_seq[..., np.newaxis]

                test_label = test_label.values
                test_label = to_categorical(test_label, num_classes=MAX_LABEL)

                test_ = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
                test_ = test_.batch(32) #(수정) 모델 자체 배치 존재? - 학습시 fit 디폴트값도 32

                inc_accuracy = evaluate(model, test_)
                print(f"Task {i} accuracy after training on Task ~{i-1}: {inc_accuracy:.4f}")




                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

        else:
            break 

# test

In [96]:
data = pd.read_pickle('mon_data.pkl')
print(data.shape)
MAX_LABEL = 30

(19000, 2)


In [99]:
# 모델 빌드 
model = DFNet.build(input_shape=(10000, 1), classes=MAX_LABEL)
# 옵티마이저 설정 
OPTIMIZER = Adamax(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
train_loop(model, OPTIMIZER, data, test_size=0.2, first_task = 19, inc_task = 5, first_epochs = 30, inc_epochs = 5, lamb=0, num_sample=30)

Epoch 1/30


c:\ProgramData\Anaconda3\envs\naivis\lib\site-packages\keras\optimizers\optimizer_v2\adamax.py:99: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


100/100 [==============================] - 7s 58ms/step - loss: 3.6932 - accuracy: 0.0691
Epoch 2/30
100/100 [==============================] - 6s 58ms/step - loss: 3.3335 - accuracy: 0.1091
Epoch 3/30
100/100 [==============================] - 6s 59ms/step - loss: 3.1769 - accuracy: 0.1262
Epoch 4/30
100/100 [==============================] - 6s 58ms/step - loss: 3.0134 - accuracy: 0.1547
Epoch 5/30
100/100 [==============================] - 6s 58ms/step - loss: 2.8939 - accuracy: 0.1650
Epoch 6/30
100/100 [==============================] - 6s 59ms/step - loss: 2.7992 - accuracy: 0.1944
Epoch 7/30
100/100 [==============================] - 6s 59ms/step - loss: 2.7054 - accuracy: 0.2150
Epoch 8/30
100/100 [==============================] - 6s 60ms/step - loss: 2.6357 - accuracy: 0.2250
Epoch 9/30
100/100 [==============================] - 6s 59ms/step - loss: 2.5806 - accuracy: 0.2428
Epoch 10/30
100/100 [==============================] - 6s 60ms/step - loss: 2.4939 - accuracy: 0.2512


In [100]:
# 모델 빌드 
model = DFNet.build(input_shape=(10000, 1), classes=MAX_LABEL)
# 옵티마이저 설정 
OPTIMIZER = Adamax(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
train_loop(model, OPTIMIZER, data, test_size=0.2, first_task = 19, inc_task = 5, first_epochs = 30, inc_epochs = 5, lamb=1, num_sample=30)

Epoch 1/30


c:\ProgramData\Anaconda3\envs\naivis\lib\site-packages\keras\optimizers\optimizer_v2\adamax.py:99: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


100/100 [==============================] - 8s 61ms/step - loss: 3.7213 - accuracy: 0.0716
Epoch 2/30
100/100 [==============================] - 6s 60ms/step - loss: 3.3375 - accuracy: 0.1078
Epoch 3/30
100/100 [==============================] - 6s 61ms/step - loss: 3.0899 - accuracy: 0.1453
Epoch 4/30
100/100 [==============================] - 6s 61ms/step - loss: 2.9301 - accuracy: 0.1725
Epoch 5/30
100/100 [==============================] - 6s 61ms/step - loss: 2.8551 - accuracy: 0.1794
Epoch 6/30
100/100 [==============================] - 6s 61ms/step - loss: 2.7756 - accuracy: 0.1931
Epoch 7/30
100/100 [==============================] - 6s 61ms/step - loss: 2.6635 - accuracy: 0.2262
Epoch 8/30
100/100 [==============================] - 6s 59ms/step - loss: 2.6293 - accuracy: 0.2194
Epoch 9/30
100/100 [==============================] - 6s 58ms/step - loss: 2.5537 - accuracy: 0.2491
Epoch 10/30
100/100 [==============================] - 6s 58ms/step - loss: 2.4874 - accuracy: 0.2606
